In [1]:
import transformers
import huggingface_hub
from huggingface_hub import notebook_login
import ipywidgets

In [2]:
notebook_login()

In [81]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader  
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
import pandas as pd 
import matplotlib.pyplot as plt

In [6]:
data = pd.read_csv("data/tweet_text_2.csv", encoding = 'cp1252') 
tweet_text = data['TWEET_TEXT']
tweet_lengths = [len(tweet) for tweet in tweet_text]


Finetuning BERT to see performance for predicting gasoline shortage

In [4]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=max, truncation=True)
model = AutoModel.from_pretrained(model_name)

tokenizing data and creating trainign an test sets 

In [102]:
data["tokenized_tweets"] = data['TWEET_TEXT'].apply(lambda x: tokenizer(x))
data = data[data.Shortage.notna()]
train_data = data.sample(frac =0.8)
test_data = data[~data.index.isin(train_data.index)]

In [103]:
test_data

,SNO,TWEET_TEXT,Shortage,Demand,Probable Shortage,tokenized_tweets
0,27751,Can someone explain to me why everyone is fill...,0.0,1.0,1.0,"[input_ids, token_type_ids, attention_mask]"
5,27836,I need gas so bad and I'm at work . Hopefully ...,0.0,1.0,1.0,"[input_ids, token_type_ids, attention_mask]"
14,28462,People kept telling me that at 4am there was g...,1.0,1.0,1.0,"[input_ids, token_type_ids, attention_mask]"
15,28511,Coral Springs area: there's gas in Wiles Rd an...,1.0,1.0,1.0,"[input_ids, token_type_ids, attention_mask]"
23,29155,I'm wasting gas driving around trying to find it,1.0,1.0,1.0,"[input_ids, token_type_ids, attention_mask]"
...,...,...,...,...,...,...
3485,821295,@politelad69 No idea. We drove through to Flor...,0.0,0.0,0.0,"[input_ids, token_type_ids, attention_mask]"
3489,822605,My gas tank is barely hanging on to life here ...,0.0,1.0,0.0,"[input_ids, token_type_ids, attention_mask]"
3490,822645,"No gas, water and ice for miles. \rOver 400k i...",1.0,1.0,1.0,"[input_ids, token_type_ids, attention_mask]"
3494,823895,Soooooo ain't no gas in Valdosta?,1.0,1.0,1.0,"[input_ids, token_type_ids, attention_mask]"


explpring the layers of bert base case

In [82]:
model.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [61]:
class Model(nn.Module):
        
    def __init__(self, num_labels):
        super().__init__()
        self.bert = model
        self.linear = nn.Linear(768,num_labels)
        self.softmax = nn.Softmax(1)
    def forward(self, x):
        bert_output = model(x)
        output = self.softmax(self.linear(bert_output))
        return output
   
num_labels = 2    
finetune_model = Model(num_labels)





Will finetune only the last layer in the first model
MODEL 1 

In [62]:
for params in finetune_model.parameters():
    params.requires_grad = False
    
for params in finetune_model.linear.parameters():
    params.requires_grad = True


Setting parameters, optimizer

In [ ]:
batch_size = 32
loss = nn.CrossEntropyLoss()
learning_rate = 1e-5
optimizer = torch.optim.Adam(finetune_model.parameters(), lr=learning_rate)

Creating a dataset and dataloader

In [ ]:
class CustomDataSet(Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        token = self.tokens[idx]
        label = self.labels[idx]

        return {"token" : token,  "label" : label}
    
train_dataset = CustomDataSet(train_data["tokenized_tweets"], train_data['Shortage'])
test_dataset = CustomDataSet(test_data["tokenized_tweets"], test_data['Shortage'])
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size)
